In this assignment, practical training issues are considered. There will be three basic issues addressed: the first describes things that need to be done prior to training a network, such as collecting and pre-processing data and selecting the network architecture; the second addresses network training itself; and the final considers post-training analysis. A case study is constructed for pattern recognition.

Assume a produce dealer has a warehouse that stores a variety of fruits and vegetables. When fruit is brought to the warehouse, various types of fruit may be mixed. The dealer wants a machine that will sort the fruit according to type. There is a conveyor belt on which the fruit is loaded. This conveyor belt passes through a set of sensors, which measure three properties of the fruit: shape, texture, and weight. These sensors are somewhat primitive. The shape sensor will output a 1 if the fruit is approximately round and a -1 if it is more elliptical. The texture sensor will output a 1 if the surface of the fruit is smooth and a -1 if it is rough. The weight sensor will output a 1 if the fruit is more than one pound and a -1 if it is less than one pound. The three sensor outputs will then be input to a neural network. The purpose of the network is to decide which kind of fruit is on the conveyor, so that the fruit can be directed to the correct storage bin. To make the problem even simpler, assume that there are only two kinds of fruit on the conveyor belt: banana and pineapple.

1. Select the network architecture and collect the pre-processing data.
2. Train the network.
3. Perform post-training and analysis.
4. Test the operation of your network by applying several different input patterns. Discuss the advantages and disadvantages of each input pattern.
5. How would you automate the training process using Python and libraries?

APA style is not required, but solid academic writing is expected. 

This assignment uses a rubric. Please review the rubric prior to beginning the assignment to become familiar with the expectations for successful completion. 

Submit a Microsoft Word document with the source code, as directed by your instructor.

Zip your document and submit to LoudCloud or GitHub, as directed by your instructor. Be sure to include the GitHub link in your Word document if submitting via GitHub.

You are required to submit this assignment to LopesWrite. Refer to the LopesWrite Technical Support articles for assistance. 

### Initialize Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# I know the assignment said we could just run 2 fruit but I ran with more to see how that would work
fruit_data = {
    "Banana": [1, 1, 1, 1, -1, 1, -1, -1, -1, 1, 1, 1, 1, 1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1],
    "Pineapple": [1, -1, -1, -1, 1, -1, 1, -1, 1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1],
    "Apple": [1, 1, 1, 1, 1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1],
    "Grapes": [1, -1, -1, -1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, 1, 1, -1, -1, -1, 1],
    "Watermelon": [1, 1, 1, 1, 1, 1, -1, -1, -1, 1, 1, -1, -1, -1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1],
    "Mango": [1, -1, -1, -1, 1, 1, 1, -1, -1, 1, 1, -1, 1, -1, 1, 1, -1, -1, 1, 1, 1, -1, -1, -1, 1]
}

### Pre-Process Data
#### Data Parameters
Hopfield networks can hold about 0.138 * n_neurons for better denoising
0.138 * n_neurons = 0.138 * 25 = 3.45 ~ 3

In [3]:
# set training parameters
n_train = 3
# len of test set
n_test = 100
# Amount of distortion (0 < distort < 1)
distort = 0.1
# No of neurons
n_neurons = 25

In [4]:
train_data = [np.array(d) for d in fruit_data.values()][:n_train]

In [5]:
test_data = []
for d in range(n_test):
    r_i = np.random.randint(0, n_train)
    base_pattern = np.array(train_data[r_i])
    noise = 1 * (np.random.random(base_pattern.shape) > distort)
    np.place(noise, noise == 0, -1)
    noisy_pattern = np.multiply(base_pattern, noise)
    test_data.append((base_pattern, noisy_pattern))

### Neural Network
Function to train the network using Hebbian learning rule

In [6]:
def train(neu, training_data):
    w = np.zeros([neu, neu])
    for data in training_data:
        w += np.outer(data, data)
    for diag in range(neu):
        w[diag][diag] = 0
    return w

In [7]:
def test(weights, testing_data):
    success = 0.0

    output_data = []

    for data in testing_data:
        true_data = data[0]
        noisy_data = data[1]
        predicted_data = retrieve_pattern(weights, noisy_data)
        if np.array_equal(true_data, predicted_data):
            success += 1.0
        output_data.append([true_data, noisy_data, predicted_data])

    return (success / len(testing_data)), output_data

In [8]:
def retrieve_pattern(weights, data, steps=10):
    res = np.array(data)

    for _ in range(steps):
        for i in range(len(res)):
            raw_v = np.dot(weights[i], res)
            if raw_v > 0:
                res[i] = 1
            else:
                res[i] = -1
    return res

In [9]:
# Train
W = train(n_neurons, train_data)

In [10]:
# Test
accuracy, out_data = test(W, test_data)

In [11]:
print("Accuracy of the network is %f" % (accuracy * 100))

Accuracy of the network is 98.000000


In [12]:
# print output classifications of test data
for v in range(len(out_data)):
    for j in range(len(list(fruit_data.keys()))):
        if all(out_data[v][2] == list(fruit_data.values())[j]):
            print(v, list(fruit_data.keys())[j])

0 Apple
1 Banana
2 Pineapple
3 Pineapple
4 Banana
5 Apple
6 Banana
7 Apple
8 Pineapple
9 Apple
10 Apple
11 Pineapple
12 Apple
13 Pineapple
14 Pineapple
15 Apple
16 Apple
17 Pineapple
18 Apple
19 Apple
20 Banana
21 Apple
22 Apple
23 Pineapple
24 Banana
25 Banana
26 Banana
27 Banana
28 Apple
29 Apple
30 Banana
31 Pineapple
32 Pineapple
33 Banana
34 Apple
36 Apple
37 Banana
38 Apple
39 Apple
40 Banana
41 Pineapple
42 Banana
43 Apple
44 Pineapple
45 Apple
46 Banana
47 Banana
48 Banana
49 Banana
50 Apple
51 Apple
52 Pineapple
53 Pineapple
54 Banana
55 Banana
56 Apple
57 Pineapple
58 Apple
59 Apple
60 Pineapple
61 Apple
62 Pineapple
63 Apple
64 Apple
65 Apple
66 Apple
67 Pineapple
68 Pineapple
69 Pineapple
70 Apple
71 Apple
72 Pineapple
73 Banana
74 Pineapple
75 Pineapple
76 Apple
77 Pineapple
78 Pineapple
79 Apple
80 Pineapple
81 Banana
82 Apple
83 Pineapple
84 Apple
85 Banana
86 Banana
87 Pineapple
88 Banana
89 Pineapple
90 Pineapple
91 Pineapple
92 Banana
93 Banana
94 Banana
95 Apple
96 A

All the nodes in a Hopfield network are both inputs and outputs, and they are fully interconnected. That is, each node is an input to every other node in the network. You can think of the links from each node to itself as being a link with a weight of 0. Inputs and outputs are binary. This technique of discrete Hopfield networks are mostly used in image recognition, with images that contain noise. In the scenario given from the assignment details, we run a similar recognition but with different fruit based on the shape, texture, and weight. These inputs are initialized as 1s and -1s. Testing data is generated as well and the classification is made and accuracy calculated. 


Some of the limitations of the Hopfield network include:

1. Memory Capacity : The number of training patterns should be less than or equal to number of neurons which means memorizing the capacity of the Hopfield network is limitied. Catastrophic forgetting occurs when there are more numbers of patterns.
2. Discrepancy limitation: The new pattern to be recognized as one of the patterns should not differ from any training pattern by more than about 25%
3. Orathogonality between patterns: The more orthigonal the training patterns, the better the recognition
4. Spurious states of attraction: Sometimes the network learns some pateerns called spurious states, which are not present in the set of training patterns
5. Weight symmetry: The weight matrix must be symmetrical in order for the network to reach an equilibrium. The symmetrical synaptic weights are not at all bilogically plausible, but they are a useful limitiation here(Limitations to using The Hopfield Network, n.d.).


References:

Limitations to using The Hopfield Network(n.d.). Retrieved from https://www.skedsoft.com/books/neural-network-fuzzy-systems/limitations-to-using-the-hopfield-network